# Usage

We'll start with a probabilistic regression example on the Boston housing dataset:

In [1]:
import sys
sys.path.append('/Users/c242587/Desktop/projects/git/ngboost')

In [2]:
from ngboost import NGBRegressor

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X, Y = load_boston(True)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

ngb = NGBRegressor().fit(X_train, Y_train)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, Y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(Y_test).mean()
print('Test NLL', test_NLL)

[iter 0] loss=3.6231 val_loss=0.0000 scale=0.5000 norm=3.3059
[iter 100] loss=3.0349 val_loss=0.0000 scale=2.0000 norm=7.1655
[iter 200] loss=2.3596 val_loss=0.0000 scale=2.0000 norm=3.7977
[iter 300] loss=1.9685 val_loss=0.0000 scale=2.0000 norm=3.0467
[iter 400] loss=1.7957 val_loss=0.0000 scale=1.0000 norm=1.3911
Test MSE 11.751316258028782
Test NLL 3.7006980539890253


Getting the estimated distributional parameters at a set of points is easy. This returns the predicted mean and standard deviation of the first five observations in the test set:

In [3]:
Y_dists[0:5].params

{'loc': array([23.46378107, 16.37767307, 23.47872484, 33.74513435, 24.77121111]),
 'scale': array([1.28837356, 1.25869559, 1.03021179, 1.01968677, 1.01327948])}

## Distributions

NGBoost can be used with a variety of distributions, broken down into those for regression (support on an infinite set) and those for classification (support on a finite set).

### Regression Distributions

| Distribution | Parameters | Implemented Scores | Reference |
| --- | --- | --- | --- |
| `Normal` | `loc`, `scale` | `LogScore`, `CRPScore` | [`scipy.stats` normal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) |
| `LogNormal` | `s`, `scale` | `LogScore`, `CRPScore` | [`scipy.stats` lognormal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lognorm.html) |
| `Exponential` | `scale` | `LogScore`, `CRPScore` | [`scipy.stats` exponential](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html) |

Regression distributions can be used through the `NGBRegressor()` constructor by passing the appropriate class as the `Dist` argument. `Normal` is the default.

In [4]:
from ngboost.distns import Exponential, Normal

X, Y = load_boston(True)
X_reg_train, X_reg_test, Y_reg_train, Y_reg_test = train_test_split(X, Y, test_size=0.2)

ngb_norm = NGBRegressor(Dist=Normal, verbose=False).fit(X_reg_train, Y_reg_train)
ngb_exp = NGBRegressor(Dist=Exponential, verbose=False).fit(X_reg_train, Y_reg_train)

There are two prediction methods for `NGBRegressor` objects: `predict()`, which returns point predictions as one would expect from a standard regressor, and `pred_dist()`, which returns a distribution object representing the conditional distribution of $Y|X=x_i$ at the points $x_i$ in the test set.

In [5]:
ngb_norm.predict(X_reg_test)[0:5]

array([32.03465906, 19.15818411, 19.30217489, 20.94209258, 18.99318666])

In [6]:
ngb_exp.predict(X_reg_test)[0:5]

array([22.28380823, 20.10027072, 17.55630199, 21.18983355, 18.11223638])

In [7]:
ngb_exp.pred_dist(X_reg_test)[0:5].params

{'scale': array([22.28380823, 20.10027072, 17.55630199, 21.18983355, 18.11223638])}

#### Survival Regression

NGBoost supports analyses of right-censored data. Any distribution that can be used for regression in NGBoost can also be used for survival analysis in theory, but this requires the implementation of the right-censored version of the appropriate score. At the moment, `LogNormal` and `Exponential` have these scores implemented. To do survival analysis, use `NGBSurvival` and pass both the time-to-event (or censoring) and event indicator vectors to  `fit()`:

In [8]:
import numpy as np
from ngboost import NGBSurvival
from ngboost.distns import LogNormal

X, Y = load_boston(True)
X_surv_train, X_surv_test, Y_surv_train, Y_surv_test = train_test_split(X, Y, test_size=0.2)

# introduce administrative censoring to simulate survival data
T_surv_train = np.minimum(Y_train, 30) # time of an event or censoring
E_surv_train = Y_train > 30 # 1 if T[i] is the time of an event, 0 if it's a time of censoring

ngb = NGBSurvival(Dist=LogNormal).fit(X_surv_train, T_surv_train, E_surv_train)

[iter 0] loss=1.2578 val_loss=0.0000 scale=4.0000 norm=2.3885
[iter 100] loss=0.5902 val_loss=0.0000 scale=2.0000 norm=0.7520
[iter 200] loss=0.3591 val_loss=0.0000 scale=4.0000 norm=0.9354
[iter 300] loss=0.2541 val_loss=0.0000 scale=2.0000 norm=0.3104
[iter 400] loss=0.1479 val_loss=0.0000 scale=4.0000 norm=0.3567


The scores currently implemented assume that the censoring is independent of survival, conditional on the observed predictors.

### Classification Distributions

| Distribution | Parameters | Implemented Scores | Reference |
| --- | --- | --- | --- |
| `k_categorical(K)` | `p0`, `p1`... `p{K-1}` | `LogScore` | [Categorical distribution on Wikipedia](https://en.wikipedia.org/wiki/Categorical_distribution) |
| `Bernoulli` | `p` | `LogScore` | [Bernoulli distribution on Wikipedia](https://en.wikipedia.org/wiki/Bernoulli_distribution) |

Classification distributions can be used through the `NGBClassifier()` constructor by passing the appropriate class as the `Dist` argument. `Bernoulli` is the default and is equivalent to `k_categorical(2)`.

In [9]:
from ngboost import NGBClassifier
from ngboost.distns import k_categorical, Bernoulli
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(True)
y[0:15] = 2 # artificially make this a 3-class problem instead of a 2-class problem
X_cls_train, X_cls_test, Y_cls_train, Y_cls_test  = train_test_split(X, y, test_size=0.2)

ngb_cat = NGBClassifier(Dist=k_categorical(3), verbose=False) # tell ngboost that there are 3 possible outcomes
_ = ngb_cat.fit(X_cls_train, Y_cls_train) # Y should have only 3 values: {0,1,2}

When using NGBoost for classification, the outcome vector `Y` must consist only of integers from 0 to K-1, where K is the total number of classes. This is consistent with the classification standards in sklearn.

`NGBClassifier` objects have three prediction methods: `predict()` returns the most likely class, `predict_proba()` returns the class probabilities, and `pred_dist()` returns the distribution object.

In [10]:
ngb_cat.predict(X_cls_test)[0:5]

array([1, 1, 1, 1, 1])

In [11]:
ngb_cat.predict_proba(X_cls_test)[0:5]

array([[7.28228735e-04, 9.99260196e-01, 1.15752705e-05],
       [7.28206439e-04, 9.99229602e-01, 4.21917010e-05],
       [7.28207212e-04, 9.99230662e-01, 4.11305541e-05],
       [7.28206439e-04, 9.99229602e-01, 4.21917010e-05],
       [7.28206439e-04, 9.99229602e-01, 4.21917010e-05]])

In [12]:
ngb_cat.pred_dist(X_cls_test)[0:5].params

{'p0': array([0.00072823, 0.00072821, 0.00072821, 0.00072821, 0.00072821]),
 'p1': array([0.9992602 , 0.9992296 , 0.99923066, 0.9992296 , 0.9992296 ]),
 'p2': array([1.15752705e-05, 4.21917010e-05, 4.11305541e-05, 4.21917010e-05,
        4.21917010e-05])}

## Scores

NGBoost supports the log score (`LogScore`, also known as negative log-likelihood) and CRPS (`CRPScore`), although each score may not be implemented for each distribution. The score is specified by the `Score` argument in the constructor.

In [13]:
from ngboost.scores import LogScore, CRPScore

NGBRegressor(Dist=Exponential, Score=CRPScore, verbose=False).fit(X_reg_train, Y_reg_train)
NGBClassifier(Dist=k_categorical(3), Score=LogScore, verbose=False).fit(X_cls_train, Y_cls_train)

NGBClassifier(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                                         max_features=None, max_leaf_nodes=None,
                                         min_impurity_decrease=0.0,
                                         min_impurity_split=None,
                                         min_samples_leaf=1,
                                         min_samples_split=2,
                                         min_weight_fraction_leaf=0.0,
                                         presort=False, random_state=None,
                                         splitter='best'),
              Dist=<class 'ngboost.distns.categorical.k_categorical.<locals>.Categorical'>,
              Score=<class 'ngboost.scores.LogScore'>, col_sample=1.0,
              learning_rate=0.01, minibatch_frac=1.0, n_estimators=500,
              natural_gradient=True,
              random_state=RandomState(MT19937) at 0x119C48D10, tol=0.0001,
              verbose=False, 

## Base Learners

NGBoost can be used with any sklearn regressor as the base learner, specified with the `Base` argument. The default is a depth-3 regression tree.

In [14]:
from sklearn.tree import DecisionTreeRegressor

learner = DecisionTreeRegressor(criterion='friedman_mse', max_depth=5)

NGBSurvival(Dist=Exponential, Score=CRPScore, Base=learner, verbose=False).fit(X_surv_train, T_surv_train, E_surv_train)

NGBSurvival(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=5,
                                       max_features=None, max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       presort=False, random_state=None,
                                       splitter='best'),
            Dist=<class 'ngboost.api.NGBSurvival.__init__.<locals>.SurvivalDistn'>,
            Score=<class 'ngboost.scores.CRPScore'>, col_sample=1.0,
            learning_rate=0.01, minibatch_frac=1.0, n_estimators=500,
            natural_gradient=True,
            random_state=RandomState(MT19937) at 0x119C48D10, tol=0.0001,
            verbose=False, verbose_eval=100)

## Other Arguments

The learning rate, number of estimators, minibatch fraction, and column subsampling are also easily adjusted:

In [15]:
ngb = NGBRegressor(n_estimators=100, learning_rate=0.01, 
             minibatch_frac=0.5, col_sample=0.5)
ngb.fit(X_reg_train, Y_reg_train)

[iter 0] loss=3.5985 val_loss=0.0000 scale=0.5000 norm=3.2693


NGBRegressor(Base=DecisionTreeRegressor(criterion='friedman_mse', max_depth=3,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=None,
                                        splitter='best'),
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.LogScore'>, col_sample=0.5,
             learning_rate=0.01, minibatch_frac=0.5, n_estimators=100,
             natural_gradient=True,
             random_state=RandomState(MT19937) at 0x119C48D10, tol=0.0001,
             verbose=True, verbose_eval=100)